In [1]:
#client_id= 21UB4FAQF0AZEJAOSR1WX1PCP0PQEBIU32IIBSBMDH0VASDO
#client_secret= 5WHV3LYBEZMIH3WYM2DUNZTNWKOABVUNYGF0A1SUS4Z4DAKL

In [1]:
import requests 
import pandas as pd 
import numpy as np 
import random 
import folium 
from geopy.geocoders import Nominatim

In [2]:
CLIENT_ID = '21UB4FAQF0AZEJAOSR1WX1PCP0PQEBIU32IIBSBMDH0VASDO' # ID
CLIENT_SECRET = '5WHV3LYBEZMIH3WYM2DUNZTNWKOABVUNYGF0A1SUS4Z4DAKL' # Secret ID
VERSION = '20190101'
LIMIT = 30

### Italian foods near to me

In [3]:
address = 'Jose Barbastre, Madrid 28017, Spain'

geolocator = Nominatim(user_agent="foursquare_agent") #A través de foursquare transforma la dirección en latitud y longitud
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.428361 -3.6516687


In [4]:
search_query = 'Italian food'
radius = 3000

In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=21UB4FAQF0AZEJAOSR1WX1PCP0PQEBIU32IIBSBMDH0VASDO&client_secret=5WHV3LYBEZMIH3WYM2DUNZTNWKOABVUNYGF0A1SUS4Z4DAKL&ll=40.428361,-3.6516687&v=20190101&query=Italian food&radius=3000&limit=30'

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f36e32331e6642aa60a33cc'},
 'response': {'venues': [{'id': '57dabf83498e545077e1e0aa',
    'name': 'Italiano Diablo Tattoo Studio',
    'location': {'address': 'Calle de Sancho Dávila, 7',
     'lat': 40.42834,
     'lng': -3.661946,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.42834,
       'lng': -3.661946}],
     'distance': 870,
     'postalCode': '28028',
     'cc': 'ES',
     'city': 'Madrid',
     'state': 'Madrid',
     'country': 'España',
     'formattedAddress': ['Calle de Sancho Dávila, 7',
      '28028 Madrid Madrid',
      'España']},
    'categories': [{'id': '4bf58dd8d48988d1de931735',
      'name': 'Tattoo Parlor',
      'pluralName': 'Tattoo Parlors',
      'shortName': 'Tattoo',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/tattoos_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1597433096',
    'hasPerk': False},
   {'id': '4c3600f066e40f47e70bcc8b',
    'name'

In [7]:
# assign relevant part of JSON to venues
venues = results['response']['venues']
#print(venues)
#tranform venues into a dataframe
dataframe = pd.json_normalize(venues)
dataframe.shape

(30, 19)

Obtener un dataframe con los nombres, categorías, dirección, codigo postal, latitud, longitud e ID.

In [8]:
dataframe.head(3)

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.crossStreet,location.neighborhood
0,57dabf83498e545077e1e0aa,Italiano Diablo Tattoo Studio,"[{'id': '4bf58dd8d48988d1de931735', 'name': 'T...",v-1597433096,False,"Calle de Sancho Dávila, 7",40.428340,-3.661946,"[{'label': 'display', 'lat': 40.42834, 'lng': ...",870,28028,ES,Madrid,Madrid,España,"[Calle de Sancho Dávila, 7, 28028 Madrid Madri...",NaN,NaN,NaN
1,4c3600f066e40f47e70bcc8b,Sabri Food Store,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",v-1597433096,False,San Emilio 4,40.429075,-3.658143,"[{'label': 'display', 'lat': 40.429075, 'lng':...",554,NaN,ES,Madrid,Madrid,España,"[San Emilio 4, Madrid Madrid, España]",NaN,NaN,NaN
2,55352493498ea9b86065290e,"Food@Life, Panaderia y Tienda Ecologica","[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",v-1597433096,False,Lopez de Hoyos 81,40.444257,-3.674358,"[{'label': 'display', 'lat': 40.44425673075245...",2612,28002,ES,Madrid,Madrid,España,"[Lopez de Hoyos 81, 28002 Madrid Madrid, España]",126681601,NaN,NaN


In [9]:
#solamente dejamos las columnas name, categories, id y todas las que comienzan con "location"
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']


In [10]:
dataframe_filtered = dataframe.loc[:, filtered_columns]

In [11]:
def get_category_type(row):
    try:
        categories_list = row['categories']    
    except:
        categories_list = row['venue.categories']    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [12]:
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
dataframe_filtered

,name,categories,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.neighborhood,id
0,Italiano Diablo Tattoo Studio,Tattoo Parlor,"Calle de Sancho Dávila, 7",40.428340,-3.661946,"[{'label': 'display', 'lat': 40.42834, 'lng': ...",870,28028,ES,Madrid,Madrid,España,"[Calle de Sancho Dávila, 7, 28028 Madrid Madri...",NaN,NaN,57dabf83498e545077e1e0aa
1,Sabri Food Store,Grocery Store,San Emilio 4,40.429075,-3.658143,"[{'label': 'display', 'lat': 40.429075, 'lng':...",554,NaN,ES,Madrid,Madrid,España,"[San Emilio 4, Madrid Madrid, España]",NaN,NaN,4c3600f066e40f47e70bcc8b
2,"Food@Life, Panaderia y Tienda Ecologica",Bakery,Lopez de Hoyos 81,40.444257,-3.674358,"[{'label': 'display', 'lat': 40.44425673075245...",2612,28002,ES,Madrid,Madrid,España,"[Lopez de Hoyos 81, 28002 Madrid Madrid, España]",NaN,NaN,55352493498ea9b86065290e
3,Neco Mediterranean Food,Buffet,Avenida de Felipe II,40.423904,-3.674889,"[{'label': 'display', 'lat': 40.42390422534488...",2029,28009,ES,Madrid,Madrid,España,"[Avenida de Felipe II, 28009 Madrid Madrid, Es...",NaN,NaN,55df0d4f498e82a12a8ec175
4,Food Truck Colmenar,Food Truck,NaN,40.425058,-3.668595,"[{'label': 'display', 'lat': 40.42505768460749...",1480,NaN,ES,NaN,NaN,España,[España],NaN,NaN,573739b5498e4cfe923d0491
5,"Lemmes the Good Food, Take Away, Delivery",Food & Drink Shop,"Padilla, 31",40.431359,-3.678308,"[{'label': 'display', 'lat': 40.431359, 'lng':...",2281,28006,ES,Madrid,Madrid,España,"[Padilla, 31 (General pardiñas), 28006 Madrid ...",General pardiñas,NaN,54f882af498e3ac3f2f03542
6,Food Truck,Food Truck,"C. General Ricardos, 125",40.424580,-3.670640,"[{'label': 'display', 'lat': 40.42458049950837...",1661,NaN,ES,Madrid,Madrid,España,"[C. General Ricardos, 125, Madrid Madrid, España]",NaN,NaN,50dc2889e4b0ab7ccc13f8ed
7,Street Food Terrace by Novotel,Restaurant,"Calle de Albacete, 1",40.441316,-3.657522,"[{'label': 'display', 'lat': 40.441316, 'lng':...",1525,28027,ES,Madrid,Madrid,España,"[Calle de Albacete, 1, 28027 Madrid Madrid, Es...",NaN,NaN,57aa3365498e4c17262d1507
8,Kyoto Gourmet Japan & Asian Food,Deli / Bodega,"C. Ayala, 65",40.427459,-3.676833,"[{'label': 'display', 'lat': 40.427459, 'lng':...",2134,NaN,ES,Madrid,Madrid,España,"[C. Ayala, 65, Madrid Madrid, España]",NaN,NaN,52d045ef498e738dd3c11c7c
9,Doki Doki,Japanese Restaurant,"Calle Villalar, 4",40.421169,-3.689797,"[{'label': 'display', 'lat': 40.42116858895287...",3328,28001,ES,Madrid,Madrid,España,"[Calle Villalar, 4, 28001 Madrid Madrid, España]",NaN,Recoletos,54243b81498e0ccd92215778


In [14]:
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]


En folium ubicar los restaurantes

In [15]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# aEl circulo rojo es el hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# los circulos azules son los restaurantes
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

Repetir los pasos anteriores con restaurantes de comida mexicana.

In [20]:
search_query = 'Mexican food'
radius = 3000

In [21]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=21UB4FAQF0AZEJAOSR1WX1PCP0PQEBIU32IIBSBMDH0VASDO&client_secret=5WHV3LYBEZMIH3WYM2DUNZTNWKOABVUNYGF0A1SUS4Z4DAKL&ll=40.428361,-3.6516687&v=20190101&query=Mexican food&radius=3000&limit=30'

In [22]:
results = requests.get(url).json()

In [25]:
# assign relevant part of JSON to venues
venues = results['response']['venues']
#print(venues)
#tranform venues into a dataframe
dataframe = pd.json_normalize(venues)
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
dataframe_filtered.head()

,name,categories,lat,lng,labeledLatLngs,distance,cc,country,formattedAddress,address,city,state,postalCode,crossStreet,neighborhood,id
0,Mexican Factory Burrito Bar,Burrito Place,40.437485,-3.686225,"[{'label': 'display', 'lat': 40.437485, 'lng':...",3099,ES,España,[España],NaN,NaN,NaN,NaN,NaN,NaN,56f2be9c498e349a052e9de9
1,Mexican's Besta Chef at Salamanca,Cocktail Bar,40.431097,-3.673894,"[{'label': 'display', 'lat': 40.4310967331239,...",1907,ES,España,[España],NaN,NaN,NaN,NaN,NaN,NaN,50d8d9d7e4b0b55bd04bd02e
2,Sabri Food Store,Grocery Store,40.429075,-3.658143,"[{'label': 'display', 'lat': 40.429075, 'lng':...",554,ES,España,"[San Emilio 4, Madrid Madrid, España]",San Emilio 4,Madrid,Madrid,NaN,NaN,NaN,4c3600f066e40f47e70bcc8b
3,"Food@Life, Panaderia y Tienda Ecologica",Bakery,40.444257,-3.674358,"[{'label': 'display', 'lat': 40.44425673075245...",2612,ES,España,"[Lopez de Hoyos 81, 28002 Madrid Madrid, España]",Lopez de Hoyos 81,Madrid,Madrid,28002,NaN,NaN,55352493498ea9b86065290e
4,"Lemmes the Good Food, Take Away, Delivery",Food & Drink Shop,40.431359,-3.678308,"[{'label': 'display', 'lat': 40.431359, 'lng':...",2281,ES,España,"[Padilla, 31 (General pardiñas), 28006 Madrid ...","Padilla, 31",Madrid,Madrid,28006,General pardiñas,NaN,54f882af498e3ac3f2f03542


In [26]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# aEl circulo rojo es el hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# los circulos azules son los restaurantes
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

Obtener la siguiente información de los primeros 3 restaurantes italianos y mexicanos: name, rating, likes y dislike y location, ordenarlos en un DataFrame.


In [28]:
dataframe_filtered.head(3)

,name,categories,lat,lng,labeledLatLngs,distance,cc,country,formattedAddress,address,city,state,postalCode,crossStreet,neighborhood,id
0,Mexican Factory Burrito Bar,Burrito Place,40.437485,-3.686225,"[{'label': 'display', 'lat': 40.437485, 'lng':...",3099,ES,España,[España],NaN,NaN,NaN,NaN,NaN,NaN,56f2be9c498e349a052e9de9
1,Mexican's Besta Chef at Salamanca,Cocktail Bar,40.431097,-3.673894,"[{'label': 'display', 'lat': 40.4310967331239,...",1907,ES,España,[España],NaN,NaN,NaN,NaN,NaN,NaN,50d8d9d7e4b0b55bd04bd02e
2,Sabri Food Store,Grocery Store,40.429075,-3.658143,"[{'label': 'display', 'lat': 40.429075, 'lng':...",554,ES,España,"[San Emilio 4, Madrid Madrid, España]",San Emilio 4,Madrid,Madrid,NaN,NaN,NaN,4c3600f066e40f47e70bcc8b


In [49]:
dataframe_filtered["id"][0]

'56f2be9c498e349a052e9de9'

In [57]:
names=[]
likes=[]
unlike=[]
location=[]
rating=[]
for i in range(7):
    venue_id=dataframe_filtered["id"][i]
    
    url='https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    names.append(result['response']['venue']['name'])
    likes.append(result['response']['venue']['likes']['count'])
    unlike.append(result['response']['venue']['dislike'])
    location.append(result['response']['venue']['location']['country'])
    try:
        rating.append(result['response']['venue']['rating'])
    except:
        rating.append("No hay rating")
df=pd.DataFrame(columns=["Names","Likes","Unlike","Location","Rating"])
df["Names"]=names
df["Likes"]=likes
df["Unlike"]=unlike
df["Location"]=location
df["Rating"]=rating

df

,Names,Likes,Unlike,Location,Rating
0,Mexican Factory Burrito Bar,7,False,España,7.5
1,Mexican's Besta Chef at Salamanca,0,False,España,No hay rating
2,Sabri Food Store,1,False,España,No hay rating
3,"Food@Life, Panaderia y Tienda Ecologica",0,False,España,No hay rating
4,"Lemmes the Good Food, Take Away, Delivery",3,False,España,No hay rating
5,Food Truck Colmenar,0,False,España,No hay rating
6,JGF fresh and fast food,2,False,España,No hay rating


## Parte 2 

Con la dirección donde vives obtener los primeros 50 venues más cercanos a tu dirección, Hacer un filtrado por la categoría que más se presenta y guardarlos en un dataframe. (Por ejemplo, si lo que más hay son Gimnasios, guardar esos gimnasios en un dataframe)


In [58]:
latitude = 40.428361
longitude = -3.6516687
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=21UB4FAQF0AZEJAOSR1WX1PCP0PQEBIU32IIBSBMDH0VASDO&client_secret=5WHV3LYBEZMIH3WYM2DUNZTNWKOABVUNYGF0A1SUS4Z4DAKL&ll=40.428361,-3.6516687&v=20190101&radius=3000&limit=30'

In [60]:
results = requests.get(url).json()
'Existen {} venues cerca de mi casa.'.format(len(results['response']['groups'][0]['items']))

'Existen 30 venues cerca de mi casa.'

In [64]:
items=results["response"]["groups"][0]['items']

In [62]:
dataframe.columns

Index(['id', 'name', 'categories', 'referralId', 'hasPerk', 'location.lat',
       'location.lng', 'location.labeledLatLngs', 'location.distance',
       'location.cc', 'location.country', 'location.formattedAddress',
       'location.address', 'location.city', 'location.state',
       'location.postalCode', 'venuePage.id', 'location.crossStreet',
       'location.neighborhood'],
      dtype='object')

In [73]:
dataframe = pd.json_normalize(items) # pasamos el JSON a dataframe

# filtramos columnas
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filtramos la categoria para cada fila
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)
df=dataframe_filtered
df.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.location.address,venue.location.crossStreet,venue.location.postalCode,venue.location.neighborhood,venue.id
0,DreamFit Ventas,Gym,40.433099,-3.656662,"[{'label': 'display', 'lat': 40.43309946547303...",676,ES,Madrid,Madrid,España,"[Madrid Madrid, España]",NaN,NaN,NaN,NaN,57752f64498e7468acd33cf5
1,Docamar,Tapas Restaurant,40.433501,-3.648133,"[{'label': 'display', 'lat': 40.43350072361024...",645,ES,Madrid,Madrid,España,"[C. Alcalá, 337 (C. de la Argentina, 8), 28027...","C. Alcalá, 337","C. de la Argentina, 8",28027,NaN,4baf340af964a520ccf03be3
2,La Fornía,Bar,40.424986,-3.654087,"[{'label': 'display', 'lat': 40.42498557232842...",428,ES,Madrid,Madrid,España,"[C. Apóstol Santiago, 6, 28017 Madrid Madrid, ...","C. Apóstol Santiago, 6",NaN,28017,NaN,4c9bacb5292a6dcbb0bbd976
3,Crossfit Singular Box,Gym / Fitness Center,40.427309,-3.662372,"[{'label': 'display', 'lat': 40.42730913124123...",914,ES,Madrid,Madrid,España,"[C. Ramón de Aguinaga, 13, 28028 Madrid Madrid...","C. Ramón de Aguinaga, 13",NaN,28028,NaN,52f28465498e14c9371d7800
4,Parque Quinta de la Fuente del Berro,Park,40.422644,-3.661108,"[{'label': 'display', 'lat': 40.42264446301398...",1022,ES,Madrid,Madrid,España,"[C. Enrique de Almonte, 28028 Madrid Madrid, E...",C. Enrique de Almonte,NaN,28028,NaN,4adcda37f964a520153c21e3


In [77]:
df.groupby(['venue.categories']).count().idxmax()[0]

'Bar'

In [78]:
df2=df[df['venue.categories']=='Bar']
df2

,venue.name,venue.categories,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.location.address,venue.location.crossStreet,venue.location.postalCode,venue.location.neighborhood,venue.id
2,La Fornía,Bar,40.424986,-3.654087,"[{'label': 'display', 'lat': 40.42498557232842...",428,ES,Madrid,Madrid,España,"[C. Apóstol Santiago, 6, 28017 Madrid Madrid, ...","C. Apóstol Santiago, 6",NaN,28017,NaN,4c9bacb5292a6dcbb0bbd976
6,Cervecería Olivares,Bar,40.437265,-3.650481,"[{'label': 'display', 'lat': 40.43726454011188...",996,ES,Madrid,Madrid,España,"[C. Virgen del Sagrario, 19 (C. Virgen de Nuri...","C. Virgen del Sagrario, 19",C. Virgen de Nuria,28027,"Ciudad Lineal, Madrid, Madrid",4c4756041ddec9280e079d32
7,La Manduca,Bar,40.434422,-3.659019,"[{'label': 'display', 'lat': 40.43442200204738...",918,ES,Madrid,Madrid,España,"[Av. Donostiarra, 2-8, 28027 Madrid Madrid, Es...","Av. Donostiarra, 2-8",NaN,28027,NaN,4bd88dbd51170f470ca033e4
8,A'Conchiña,Bar,40.431764,-3.650525,"[{'label': 'display', 'lat': 40.43176391882853...",391,ES,Madrid,Madrid,España,"[C. Benidorm, 32, 28017 Madrid Madrid, España]","C. Benidorm, 32",NaN,28017,NaN,4e24070cb0fbdf9ba7c32414
13,El Callejón de Álvarez Gato,Bar,40.436425,-3.644408,"[{'label': 'display', 'lat': 40.43642507036786...",1088,ES,Madrid,Madrid,España,"[C. Elfo, 123, 28027 Madrid Madrid, España]","C. Elfo, 123",NaN,28027,NaN,4b8c0dc0f964a52009bb32e3


Del dataframe anterior obtener name, rating y likes y seleccionar al que tiene más rating y más likes. 

In [79]:
df["venue.location.lat"][0]

40.43309946547303

In [ ]:
for i in range(len(df)):
    latitude= df["venue.location.lat"][i]
    longitude= df["venue.location.lng"][i]
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
    

En un Mapa mostrar los venues anteriores. (solamente los venues que comparten la misma categoría mayoritaria)